In [14]:
import os
import json
import cv2
import pandas as pd
import numpy as np
import torch.utils.data as data
from pycocotools.coco import COCO
from albumentations.pytorch import ToTensorV2
import albumentations as A

dataset_path = '../input/data'
anns_file_path = dataset_path + '/' + 'train.json'

# Read annotations
with open(anns_file_path, 'r') as f:
    dataset = json.loads(f.read())

categories = dataset['categories']
anns = dataset['annotations']
imgs = dataset['images']
nr_cats = len(categories)
nr_annotations = len(anns)
nr_images = len(imgs)

# Load categories and super categories
cat_names = []
super_cat_names = []
super_cat_ids = {}
super_cat_last_name = ''
nr_super_cats = 0
for cat_it in categories:
    cat_names.append(cat_it['name'])
    super_cat_name = cat_it['supercategory']
    # Adding new supercat
    if super_cat_name != super_cat_last_name:
        super_cat_names.append(super_cat_name)
        super_cat_ids[super_cat_name] = nr_super_cats
        super_cat_last_name = super_cat_name
        nr_super_cats += 1

cat_histogram = np.zeros(nr_cats, dtype=int)
for ann in anns:
    cat_histogram[ann['category_id']] += 1

df = pd.DataFrame({'Categories': cat_names, 'Number of annotations': cat_histogram})
df = df.sort_values('Number of annotations', 0, False)
sorted_temp_df = df.sort_index()

# background = 0 에 해당되는 label 추가 후 기존들을 모두 label + 1 로 설정
sorted_df = pd.DataFrame(["Backgroud"], columns=["Categories"])
sorted_df = sorted_df.append(sorted_temp_df, ignore_index=True)
category_names = list(sorted_df.Categories)


class NormalizedAugmentation:
    '''
    기본값인 mean 과 std를 사용한 Augmentation
    '''
    def __init__(self, mean=(0.5, 0.5, 0.5), std=(0.25, 0.25, 0.25), **args):
        self.transform = A.Compose([
            A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ])

    def __call__(self, image):
        return self.transform(image)

class NewAugmentation:
    '''
    기본값인 mean 과 std를 사용한 Augmentation
    '''

    def __init__(self, mean=(0.5, 0.5, 0.5), std=(0.25, 0.25, 0.25), **args):
        self.transform = A.Compose([
            A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(),
        ])
        self.train_transform = A.Compose([
            A.HorizontalFlip(p=0.3),
            A.Rotate(p=0.3, limit=45),
            A.Cutout(num_holes=4, max_h_size=20, max_w_size=20),
            A.CLAHE(),
            A.RandomBrightnessContrast(p=0.3),
            A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2()
        ])
#A.Cutout(num_holes=4, max_h_size=20, max_w_size=20),


    def __call__(self, image,mode):
        if mode=='train':
            return self.train_transform(image)
        elif mode=='val':
            return self.transform(image)
        elif mode=='test':
            return self.transform(image)

class PseudoTrainset(data.Dataset):
    """COCO format"""
    def __init__(self, data_dir,mode = 'train'):
        super().__init__()
        self.mode = mode
        self.transform = None
        self.coco = COCO(data_dir)
        self.dataset_path = '../input/data/'
        self.category_names = ['Backgroud', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        
        self.pseudo_imgs = np.load(self.dataset_path+'pseudo_imgs_path.npy')
        self.pseudo_masks = sorted(glob.glob(self.dataset_path+'pseudo_masks/*.npy'))
        
    def __getitem__(self, index: int):
        
        ### Train data ###
        if (index < len(self.coco.getImgIds())):
            image_id = self.coco.getImgIds(imgIds=index)
            image_infos = self.coco.loadImgs(image_id)[0]

            images = cv2.imread(self.dataset_path+image_infos['file_name'])
            images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
            images /= 255.0
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)
            
            ###  mask 생성  ###
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = self.category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)

        ### Pseudo data ###
        else:
            index -= len(self.coco.getImgIds())
            images = cv2.imread(self.dataset_path+self.pseudo_imgs[index])
            images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
            masks = np.load(self.pseudo_masks[index])
            
        ###  augmentation ###
        masks = masks.astype(np.float32)
        if self.transform is not None:
                if self.mode=='train':
                    transformed = self.transform.train_transform(image=images, mask=masks)
                    images = transformed["image"]
                    masks = transformed["mask"]
                elif self.mode=='val':
                    transformed = self.transform.transform(image=images, mask=masks)
                    images = transformed["image"]
                    masks = transformed["mask"]
                elif self.mode=='test':
                    transformed = self.transform.transform(image=images)
                    images = transformed["image"]
            
        return images, masks
    
    def set_transform(self, transform):
        '''
        :param transform: 우리가 원하는 transform으로 dataset의 transform을 설정해준다
        '''
        self.transform = transform
    
    def __len__(self):
        return len(self.coco.getImgIds())+len(self.pseudo_imgs)

class TrashDataset(data.Dataset):
    def __init__(self, data_dir, mode='train'):
        self.transform = None
        self.mode = mode
        self.data_dir = data_dir
        self.coco = COCO(self.data_dir)

    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)

        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            annss = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(annss)):
                className = self.get_classname(annss[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(annss[i]) * pixel_value, masks)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                if self.mode=='train':
                    transformed = self.transform.train_transform(image=images, mask=masks)
                    images = transformed["image"]
                    masks = transformed["mask"]
                elif self.mode=='val':
                    transformed = self.transform.transform(image=images, mask=masks)
                    images = transformed["image"]
                    masks = transformed["mask"]

            return images, masks, image_infos

        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform.transform(image=images)
                images = transformed["image"]

            return images, image_infos

    def get_classname(self, classID, cats):
        for i in range(len(cats)):
            if cats[i]['id'] == classID:
                return cats[i]['name']
        return "None"

    def set_transform(self, transform):
        '''
        :param transform: 우리가 원하는 transform으로 dataset의 transform을 설정해준다
        '''
        self.transform = transform

    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [15]:
import numpy as np

def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(n_class * label_true[mask].astype(int) + label_pred[mask],
                        minlength=n_class ** 2).reshape(n_class, n_class)
    return hist


def label_accuracy_score(hist):
    """
    Returns accuracy score evaluation result.
      - [acc]: overall accuracy
      - [acc_cls]: mean accuracy
      - [mean_iu]: mean IU
      - [fwavacc]: fwavacc
    """
    acc = np.diag(hist).sum() / hist.sum()
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)

    with np.errstate(divide='ignore', invalid='ignore'):
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)

    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc


def add_hist(hist, label_trues, label_preds, n_class):
    """
        stack hist(confusion matrix)
    """

    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)

    return hist

In [16]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

In [17]:
import torch.optim.lr_scheduler as lr_scheduler
import math
class CosineAnnealingWarmUpRestart(lr_scheduler._LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestart, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr) * self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (
                        1 + math.cos(math.pi * (self.T_cur - self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch

        self.eta_max = self.base_eta_max * (self.gamma ** self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [ ]:

import glob
import os
import random
import re
from importlib import import_module
from pathlib import Path
import numpy as np
from tqdm import tqdm
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
from adamp import AdamP

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def increment_path(path, exist_ok=False):
    """ Automatically increment path, i.e. runs/exp --> runs/exp0, runs/exp1 etc.

    Args:
        path (str or pathlib.Path): f"{model_dir}/{args.name}".
        exist_ok (bool): whether increment path (increment if False).
    """
    path = Path(path)
    if (path.exists() and exist_ok) or (not path.exists()):
        return str(path)
    else:
        dirs = glob.glob(f"{path}*")
        matches = [re.search(rf"%s(\d+)" % path.stem, d) for d in dirs]
        i = [int(m.groups()[0]) for m in matches if m]
        n = max(i) + 1 if i else 2
        return f"{path}{n}"


def collate_fn(batch):
    return tuple(zip(*batch))


def train(data_dir, model_dir):
    train_path = data_dir + '/train_all.json'
    val_path = data_dir + '/val.json'

    seed_everything(42)
    save_dir = './'+increment_path(os.path.join(model_dir, 'oneloss+alldata'))
    os.makedirs(save_dir)
    # -- settings
    use_cuda = torch.cuda.is_available()
    print("PyTorch version:[%s]." % (torch.__version__))
    GPU_NUM = 0 # 원하는 GPU 번호 입력
    device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
    print("device:[%s]." % (device))

    # -- dataset  # default: BaseAugmentation
    train_dataset = PseudoTrainset(
        data_dir=train_path,
        mode='train'
    )
    val_dataset = TrashDataset(
        data_dir=val_path,
        mode='val'
    )

    # -- augmentation  # default: BaseAugmentation
    transform = NewAugmentation()
    train_dataset.set_transform(transform)
    val_dataset.set_transform(transform)

    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        num_workers=4,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        num_workers=4,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=True
    )
    
    # -- model  # default: BaseModel
    #model_path = './model/exp/best.pth'
    #checkpoint = torch.load(model_path, map_location=device)
    model = smp.DeepLabV3Plus(
        encoder_name="timm-efficientnet-b5",
        encoder_weights="noisy-student",
        in_channels=3,
        classes=12,
    ).to(device)
    #model.load_state_dict(checkpoint)
    
    wandb.watch(model)
    criterion1 = FocalLoss(gamma=0.5)
    #criterion2 = nn.CrossEntropyLoss(weight=normed_weights)
    #criterion2 = smp.losses.DiceLoss(mode='multiclass')
    criterion2 = smp.losses.SoftCrossEntropyLoss(smooth_factor=0.1)
    optimizer = AdamP(
        model.parameters(),
        lr=5e-6,
        betas=(0.9, 0.999),
        weight_decay=1e-4,
        eps=1e-6
    )
    # optimizer = AdamP(model.parameters(), lr=args.lr, betas=(0.9, 0.999), weight_decay=1e-2)
    # optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr, weight_decay=1e-6)
    #scheduler = StepLR(optimizer, 1, gamma=0.7)
    scheduler = CosineAnnealingWarmUpRestart(optimizer, T_0=4, T_mult=1, eta_max=2e-4,  T_up=1, gamma=0.75)
    # -- logging
        
        
        
    best_mIoU = 0
    best_val_loss = np.inf
    for epoch in range(1,21):
        # train loop
        model.train()
        loss_value = 0
        for idx, (images, masks) in enumerate(tqdm(train_loader)):
            images = torch.stack(images)  # (batch, channel, height, width)
            masks = torch.stack(masks).long()
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            #loss1 = criterion1(outputs, masks)
            loss = criterion2(outputs, masks)
            #loss = 0.75*loss1+0.25*loss2
            loss.backward()
            optimizer.step()

            loss_value += loss.item()
            if (idx + 1) % 20 == 0:
                train_loss = loss_value / 20
                current_lr = get_lr(optimizer)
                print(
                    f"Epoch[{epoch}/{20}]({idx + 1}/{len(train_loader)}) || "
                    f"training loss {train_loss:4.4} || lr {current_lr}"
                )
                wandb.log({"train loss": train_loss})
                loss_value = 0
        hist = np.zeros((12, 12))
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            for idx, (images, masks, _) in enumerate(val_loader):
                images = torch.stack(images)  # (batch, channel, height, width)
                masks = torch.stack(masks).long()
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                #loss1 = criterion1(outputs, masks)
                loss = criterion2(outputs, masks)
                #loss = 0.75*loss1+0.25*loss2
                val_loss_items.append(loss)
                outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
                hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=12)
            val_loss = np.sum(val_loss_items) / len(val_loader)
            best_val_loss = min(best_val_loss, val_loss)
            _, _,mIoU,_ = label_accuracy_score(hist)
            wandb.log({
                        "Test mIoU": mIoU,
                        "Test Loss": val_loss})
            if mIoU > best_mIoU:
                print(f"New best model for val accuracy : {mIoU:4.2%}! saving the best model..")
                torch.save(model.state_dict(), f"{save_dir}/best.pth")
                best_mIoU = mIoU
            torch.save(model.state_dict(), f"{save_dir}/last.pth")
            print(
                f"[Val] mIoU : {mIoU:4.2%}, loss: {val_loss:4.2} || "
                f"best mIoU : {best_mIoU:4.2%}, best loss: {best_val_loss:4.2}"
            )
        scheduler.step()
        # val loop


import wandb
data_dir = os.environ.get('SM_CHANNEL_TRAIN', '../input/data')
model_dir = os.environ.get('SM_MODEL_DIR', './model')
wandb.init(project='chowon', entity='pstage12')
wandb.run.name = 'oneloss+alldata'
train(data_dir, model_dir)

In [9]:

import numpy as np
import albumentations as A
import pandas as pd 
import segmentation_models_pytorch as smp
from tqdm import tqdm

use_cuda = torch.cuda.is_available()
print("PyTorch version:[%s]." % (torch.__version__))
device = torch.device('cuda' if use_cuda else 'cpu')
print("device:[%s]." % (device))
    
model_path = './model/Augmentation_bestparam_nocontrast/best.pth'
dataset_path = '../input/data'
test_path = dataset_path + '/test.json'


def collate_fn(batch):
    return tuple(zip(*batch))


# best model 불러오기
batch_size = 9
checkpoint = torch.load(model_path, map_location=device)
model = smp.DeepLabV3Plus(
    encoder_name="timm-efficientnet-b5",
    encoder_weights="noisy-student",
    in_channels=3,
    classes=12
).to(device)
model.load_state_dict(checkpoint)
test_dataset = TrashDataset(data_dir=test_path, mode='test')  # default: NormalizedAugmentation
transform = NormalizedAugmentation()
test_dataset.set_transform(transform)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          num_workers=4,
                                          collate_fn=collate_fn)


def test(model, data_loader, device):
    size = 256
    transformer = A.Compose([A.Resize(256, 256)])
    tta = A.HorizontalFlip(p=1)
    print('Start prediction.')
    model.eval()
    file_name_list = []
    preds_array = np.empty((0, size * size), dtype=np.long)

    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(tqdm(data_loader)):
            
            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transformer(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)

            oms = oms.reshape([oms.shape[0], size * size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]

    return file_names, preds_array


submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append(
        {"image_id": file_name, "PredictionString": ' '.join(str(e) for e in string.tolist())},
        ignore_index=True)

# submission.csv로 저장
submission.to_csv("./submission/hahahoho.csv", index=False)


PyTorch version:[1.4.0].
device:[cuda].



  0%|          | 0/93 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Start prediction.



100%|██████████| 93/93 [13:06<00:00,  8.45s/it]


End prediction.
